# Operaciones basicas con groupy apply

El objetivo de este notebook es mostrar algunas operaciones sencillas con el método groupby combinado con estadisticas que agregen información o con procesos más específicos. 

In [64]:
import pandas as pd
import numpy as np
from scipy.stats import iqr


In [59]:
#creamos un data set de ejemplo con notas de examenes
dt = pd.DataFrame({'alumnos':['Juan','Pedro','Diego','Ana','Camila','Veronica'],
                   'sexo':['varon'] * 3 + ['mujer'] * 3,
                  'notas':[1,2,3,8,1,7]})
dt

,alumnos,sexo,notas
0,Juan,varon,1
1,Pedro,varon,2
2,Diego,varon,3
3,Ana,mujer,8
4,Camila,mujer,1
5,Veronica,mujer,7


## Aggregate

In [61]:
#existen funciones integradas que ya hacen algunos procesos por nosotros, como el promedio
dt.groupby('sexo').mean()

,notas
sexo,
mujer,5.333333
varon,2.000000


In [68]:
# es lo mismo que correr una funcion que agrege estos datos
dt.groupby('sexo').agg([np.mean])

,notas
,mean
sexo,
mujer,5.333333
varon,2.000000


In [70]:
#no todas las funciones se pueden utilizar asi, como por ejemplo el rango intercuartilico del paquete Scipy
dt.groupby('sexo').iqr()

AttributeError: 'DataFrameGroupBy' object has no attribute 'iqr'

In [71]:
#pero podemos agregar más de una funcion
dt.groupby('sexo').agg([np.mean,iqr])

notas     
           mean  iqr
sexo                
mujer  5.333333  3.5
varon  2.000000  1.0

In [72]:
#existe una funcion util que es describe con una bateria de indicadores
dt.groupby('sexo').describe()

notas                                             
      count      mean       std  min  25%  50%  75%  max
sexo                                                    
mujer   3.0  5.333333  3.785939  1.0  4.0  7.0  7.5  8.0
varon   3.0  2.000000  1.000000  1.0  1.5  2.0  2.5  3.0

## Apply

Apply es un metodo que aplica funciones que no necesariamente deben agregar. Pueden hacerlo o no, lo que nos deja manipular los datos de manera más customizada. 

In [74]:
#podemos obtener el promedio obteniendo el promedio de la columna para cada grupo 
def promedio(sub_dt):
    return sub_dt.notas.mean()

dt.groupby('sexo').apply(promedio)

sexo
mujer    5.333333
varon    2.000000
dtype: float64

In [75]:
#pero tambien nos realizar operaciones que no agregen como seleccionar elementos
def obtener_primer_elemento(sub_dt):
    return sub_dt.iloc[0,:]

dt.groupby('sexo').apply(obtener_primer_elemento)

,alumnos,sexo,notas
sexo,,,
mujer,Ana,mujer,8
varon,Juan,varon,1


In [76]:
#podemos hacer operaciones para cada subset de datos, como normalizar para el promedio y desvio del grupo y no de la poblacion
def puntaje_z(sub_dt):
    return (sub_dt.notas - sub_dt.notas.mean()) /sub_dt.notas.std()

#deja el index
dt.groupby('sexo').apply(puntaje_z)

sexo    
mujer  3    0.704361
       4   -1.144586
       5    0.440225
varon  0   -1.000000
       1    0.000000
       2    1.000000
Name: notas, dtype: float64

In [78]:
#asi seria la normalizacion para la poblacion
(dt.notas - dt.notas.mean()) /dt.notas.std()

0   -0.866703
1   -0.541689
2   -0.216676
3    1.408392
4   -0.866703
5    1.083378
Name: notas, dtype: float64

In [79]:
#apply puede aplicarse no solo agrupada, sino para todo un dataset, por fila
jubilar = pd.DataFrame({'alumnos':['Juan','Pedro','Diego','Ana','Camila','Veronica'],
                   'sexo':['varon'] * 3 + ['mujer'] * 3,
                  'edad':[60,70,65,55,70,65]})
jubilar

,alumnos,sexo,edad
0,Juan,varon,60
1,Pedro,varon,70
2,Diego,varon,65
3,Ana,mujer,55
4,Camila,mujer,70
5,Veronica,mujer,65


In [80]:
def puede_jubilarse(fila):
    if fila.sexo == 'varon':
        edad_jubilatoria = 65
    elif fila.sexo == 'mujer':
        edad_jubilatoria = 60
    else:
        raise ValueError('Problema con el sexo')
        
    if fila.edad >= edad_jubilatoria:
        return 'Si'
    else:
        return 'No'

    
    
    

In [81]:
jubilar.apply(puede_jubilarse,axis=1)

0    No
1    Si
2    Si
3    No
4    Si
5    Si
dtype: object

In [82]:
jubilar['puede'] = jubilar.apply(puede_jubilarse,axis=1)
jubilar

,alumnos,sexo,edad,puede
0,Juan,varon,60,No
1,Pedro,varon,70,Si
2,Diego,varon,65,Si
3,Ana,mujer,55,No
4,Camila,mujer,70,Si
5,Veronica,mujer,65,Si


In [29]:
import seaborn as sns
tips = sns.load_dataset('tips')
tips.head()

Tambien podemos crear una funcion con parametros que si no se explicitan van a tomar valores por defecto (`5` y `tip`) y l

In [33]:
def top(dt,n=5,column = 'tip'):
    return dt.sort_values(by=column)[-n:]

In [36]:
tips.groupby('smoker').apply(top,n=1)

,,total_bill,tip,sex,smoker,day,time,size
smoker,,,,,,,,
Yes,170,50.81,10.0,Male,Yes,Sat,Dinner,3
No,212,48.33,9.0,Male,No,Sat,Dinner,4


In [83]:
tips.groupby('smoker').apply(top,n=2,column='size')

total_bill  tip     sex smoker   day    time  size
smoker                                                        
Yes    216       28.15  3.0    Male    Yes   Sat  Dinner     5
       187       30.46  2.0    Male    Yes   Sun  Dinner     5
No     156       48.17  5.0    Male     No   Sun  Dinner     6
       125       29.80  4.2  Female     No  Thur   Lunch     6